Подключение необходимых библиотек

In [ ]:
from datetime import timedelta
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import os
import yaml
import datetime
import torch
import warnings
import glob
import sys
import matplotlib.pyplot as plt
import seaborn as sn

Константа, количество кадров в секунду

In [ ]:
SAVING_FRAMES_PER_SECOND = 2

  format_timedelta
    
    Служебная функция для классного форматирования объектов timedelta
    исключая микросекунды и сохраняя миллисекунды

  get_saving_frames_durations

    Функция, которая возвращает список длительностей, в которые следует сохранять кадры.

In [ ]:
def format_timedelta(td):
    result = str(td)
    try:
        result, ms = result.split(".")
    except ValueError:
        return result + ".00".replace(":", "-")
    ms = int(ms)
    ms = round(ms / 1e4)
    return f"{result}.{ms:02}".replace(":", "-")


def get_saving_frames_durations(cap, saving_fps):
    s = []
    clip_duration = cap.get(cv2.CAP_PROP_FRAME_COUNT) / cap.get(cv2.CAP_PROP_FPS)
    for i in np.arange(0, clip_duration, 1 / saving_fps):
        s.append(i)
    return s

Для анализа изображений используется нейросеть Yolo v5

Подгружается готовая, обученная нейросеть

In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True, device='cpu')

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-11-25 Python-3.7.15 torch-1.12.1+cu113 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


Видео разбивается на кадры, каждый кадр обрабатывается нейронной сетью

После обработки поверх 1 кадра наносится новый слой обозначений

video_file - путь к файлу

color - цвет

min, max - пределы значений для составления тепловой карты  

log - вывод в консоль информации о людях на кадре

save_all_frames - флаг, если True то полученные промежуточные результаты будут также сохраняться

После выполнения функция возвращает кадр на котором отмечены места скопления людей

In [ ]:
def detect(video_file, color = (0, 0, 255), vmax = 125, vmin = 0, log = False):
    frame_to_draw = None
    f = True
    filename, _ = os.path.splitext(video_file)
    filename += "-results"
    if not os.path.isdir(filename):
        os.mkdir(filename)
    cap = cv2.VideoCapture(video_file)
    fps = cap.get(cv2.CAP_PROP_FPS)
    saving_frames_per_second = min(fps, SAVING_FRAMES_PER_SECOND)
    saving_frames_durations = get_saving_frames_durations(cap, saving_frames_per_second)
    count = 0
    layer = 0
    map_data = []
    while True:
        is_read, frame = cap.read()
        if(f == True):
              frame_to_draw = frame.copy()
              f = False
        if not is_read:
            break
        frame_duration = count / fps
        try:
            closest_duration = saving_frames_durations[0]
        except IndexError:
            break
        if frame_duration >= closest_duration:
            frame_duration_formatted = format_timedelta(timedelta(seconds=frame_duration))
            path = filename+'/'+f"frame{frame_duration_formatted}.jpg";
            if(f == True):
              frame_to_draw = frame.copy()
              height, width, channels = frame_to_draw.shape
              #map_data = [[0 for j in int(height)] for i in int(width)]
              map_data = [0]*int(width)
              for i in range(0, int(width)):
                map_data[i] = [0]*int(height)
              f = False
            try:
                saving_frames_durations.pop(0)
            except IndexError:
                pass
            results = model(frame)
            results = model(frame).pandas().xyxy[0]
            results = results[results['name'] == 'person']
            if(len(results) > 0):
              if(log):
                print('Количество людей на кадре: ' + str(len(results)))
                print(results)
              for i in results['xmin'].keys():
                for j in range(int(results["xmin"][i]), int(results["xmax"][i])):
                  for k in range(int(results["ymin"][i]), int(results["ymax"][i])):
                    if(map_data[j][k] >= vmin):
                      if(map_data[j][k] < vmax):
                        map_data[j][k] = map_data[j][k] + 1
              results = results.to_numpy()
              for i in results:
                frame1 = frame_to_draw.copy()
                cv2.rectangle(frame1, (int(i[0]), int(i[1])), (int(i[2]), int(i[3])), color, -1)
              if(log):
                print(f"frame: {frame_duration_formatted}")
        count += 1
    cv2.imwrite(os.path.join(filename, f"result.jpg"), frame_to_draw)
    map_data = np.fliplr(map_data)
    map_data = np.rot90(map_data)

    return np.array(map_data)

Вызов функции detect и сохранение итогового изображения

In [ ]:
result = detect('/content/drive/MyDrive/1/1.mp4', (0, 0, 255), 125, 0, True)

KeyboardInterrupt: ignored

Построение тепловой карты с помощью библиотеки Seaborn и очистка plt

In [ ]:
plt.close()
img1 = plt.imread("/content/drive/MyDrive/1/1-opencv/frame0-00-20.33.jpg")
#cv2.resize(img1, [432, 288])
hm = sn.heatmap(result,cmap="Blues")

hm.figure.savefig('result1')

Вывод результата

In [ ]:
#plt.savefig('result.png')
result = cv2.imread('result1.png', cv2.IMREAD_COLOR)
#cv2_imshow(result)
hm.imshow(img1)
plt.show()